In [1]:
!pip freeze | grep scikit-learn

"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [15]:
year = 2021
month = 2

In [16]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [17]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [18]:
y_pred.mean()

16.191691679979066

In [19]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [21]:
df_pred = pd.DataFrame()

In [22]:
df_pred['ride_id'] = df['ride_id']
df_pred['prediction'] = y_pred

In [26]:
df_pred.to_parquet(path = './predictions.parquet', engine = 'pyarrow', index=False, compression='None')